# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [127]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [128]:
# import cities data from WeatherPy assignment
filename = "Resources/cities.csv"
cities_data = pd.read_csv(filename)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [129]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# plot humidity heatmap
fig = gmaps.figure(map_type="HYBRID")

# create input data for heatmap
locations = cities_data[["latitude", "longitude"]]
humidity = cities_data["humidity"]

# create heat layer
heat_layer = gmaps.heatmap_layer(
    locations, 
    weights=humidity, 
    dissipating=False, 
    max_intensity=250, 
    point_radius=3
)

# add layer
fig.add_layer(heat_layer)

# display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [130]:
# filter cities dataset to maximum 10 cities based on weather
cities_filter1 = cities_data.loc[(cities_data["temperature"] >= 50) & (cities_data["temperature"] <= 75)]
cities_filter2 = cities_filter1.loc[cities_filter1["humidity"] < 32]
cities_filter3 = cities_filter2.loc[cities_filter2["cloudiness"] <= 10]
cities_filter = cities_filter3

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [146]:
# store into variable named hotel_df
hotel_df = cities_filter

# add a "hotel name" column to hotel_df
hotel_df["hotel name"] = ""

# set params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key
}

# Use the lat/lng to find hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["latitude"]
    lng = row["longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    hotel_data = response.json()

    # store hotel name in hotel_df
    try:
        hotel_df.loc[index, "hotel name"] = hotel_data["results"][1]["name"]
    except (KeyError, IndexError):
        print("Information not found... skipping.")



In [147]:
hotel_df.head(10)

,city,latitude,longitude,temperature,humidity,cloudiness,wind speed,country,date,hotel name
223,dalsingh sarai,25.6667,85.8333,72.37,30,0,6.11,IN,1611644457,Gramin Bank
233,nalagarh,31.0500,76.7167,68.20,28,0,6.11,IN,1611644458,Fort Nalagarh
236,pauri,30.1500,78.7833,62.82,27,0,5.19,IN,1611644459,Jatra Heights
242,sanaur,30.3025,76.4553,67.68,31,0,7.90,IN,1611644460,Pepsu International Public School
244,sitamau,24.0167,75.3500,69.22,24,0,8.70,IN,1611644460,Jawahar Navodaya Vidyalaya Laduna
283,jalu,29.0331,21.5482,60.35,31,0,10.92,LY,1611644466,Trade and Development Bank Branch gallo
380,chor,25.5167,69.7667,74.68,16,0,3.47,PK,1611644485,Ali Nawaz Nohri Autoz Chhor
381,lodhran,29.5405,71.6336,72.90,17,0,6.67,PK,1611644485,Muhammadia Clinic And Maternity Home
383,tandlianwala,31.0333,73.1333,70.20,20,0,5.28,PK,1611644485,Ahsanabba Samedicare
516,hobart,-42.8794,147.3294,66.20,26,0,21.85,AU,1611644404,St Ives Hobart Accommodation Tasmania


In [151]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [161]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(
    locations, info_box_content=hotel_info
)

# add layer
fig.add_layer(hotel_layer)

# display figure
fig

Figure(layout=FigureLayout(height='420px'))